**Подготовка окружения**

Установка библиотеки, осуществляющей выбор минимально оптимальных признаков (поиск наименьшего релевантного подмножества функций для данной задачи машинного обучения).



In [1]:
! pip install mrmr_selection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.2 MB/s eta 0:00:00


Подключение гугл-диска, на котором хранятся данные

In [2]:
! pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Включение опции "показывать все столбцы"

In [5]:
pd.set_option('display.max_columns', None)

Подключение библиотек

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from mrmr import mrmr_classif
import pickle
import sklearn.metrics
import numpy as np
import pandas as pd

Загрузка данных датасета SWELL с гугл-диска

In [6]:
# data source - https://www.kaggle.com/datasets/qiriro/stress
swell_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/combined-swell-classification-hrv-train-dataset.csv")
swell_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/combined-swell-classification-hrv-test-dataset.csv")

In [7]:
# Присвоение состояниям субъектов метки "Condition Label": 1 - стресс, 0 - не стресс
label_swell_train = pd.DataFrame(swell_train['Condition Label'].map({0: 0, 1: 1, 2: 1}))
label_swell_test = pd.DataFrame(swell_test['Condition Label'].map({0: 0, 1: 1, 2: 1}))
label_swell_train

,Condition Label
0,1
1,1
2,1
3,1
4,1
...,...
313305,0
313306,0
313307,1
313308,0


In [8]:
# Показать все столбцы
swell_train.columns

Index(['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR',
       'pNN25', 'pNN50', 'SD1', 'SD2', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR',
       'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'VLF_PCT',
       'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF',
       'HF_LF', 'sampen', 'higuci', 'condition', 'subject_id', 'MEAN_RR_LOG',
       'MEAN_RR_SQRT', 'TP_SQRT', 'MEDIAN_REL_RR_LOG', 'RMSSD_REL_RR_LOG',
       'SDSD_REL_RR_LOG', 'VLF_LOG', 'LF_LOG', 'HF_LOG', 'TP_LOG', 'LF_HF_LOG',
       'RMSSD_LOG', 'SDRR_RMSSD_LOG', 'pNN25_LOG', 'pNN50_LOG', 'SD1_LOG',
       'KURT_YEO_JONSON', 'SKEW_YEO_JONSON', 'MEAN_REL_RR_YEO_JONSON',
       'SKEW_REL_RR_YEO_JONSON', 'LF_BOXCOX', 'HF_BOXCOX', 'SD1_BOXCOX',
       'KURT_SQUARE', 'HR_SQRT', 'MEAN_RR_MEAN_MEAN_REL_RR', 'SD2_LF', 'HR_LF',
       'HR_HF', 'HF_VLF', 'NasaTLX class', 'Condition Label', 'NasaTLX Label'],
      dtype='object')

In [9]:
# Идентифицировать все уникальные идентификаторы субъектов в данных SWELL
subject_id_swell_train=swell_train['subject_id']
subject_id_swell_test=swell_test['subject_id']
# Удалить все столбцы, которые не будут использоваться в качестве входных данных для обучения/тестирования модели
swell_train=swell_train.drop(columns=['subject_id','sampen', 'higuci','condition', 'MEAN_RR_LOG',
       'MEAN_RR_SQRT', 'TP_SQRT', 'MEDIAN_REL_RR_LOG', 'RMSSD_REL_RR_LOG',
       'SDSD_REL_RR_LOG', 'VLF_LOG', 'LF_LOG', 'HF_LOG', 'TP_LOG', 'LF_HF_LOG',
       'RMSSD_LOG', 'SDRR_RMSSD_LOG', 'pNN25_LOG', 'pNN50_LOG', 'SD1_LOG',
       'KURT_YEO_JONSON', 'SKEW_YEO_JONSON', 'MEAN_REL_RR_YEO_JONSON',
       'SKEW_REL_RR_YEO_JONSON', 'LF_BOXCOX', 'HF_BOXCOX', 'SD1_BOXCOX',
       'KURT_SQUARE', 'HR_SQRT', 'MEAN_RR_MEAN_MEAN_REL_RR', 'SD2_LF', 'HR_LF',
       'HR_HF', 'HF_VLF', 'NasaTLX class', 'Condition Label', 'NasaTLX Label'])
swell_test=swell_test.drop(columns=['subject_id','sampen', 'higuci','condition', 'MEAN_RR_LOG',
       'MEAN_RR_SQRT', 'TP_SQRT', 'MEDIAN_REL_RR_LOG', 'RMSSD_REL_RR_LOG',
       'SDSD_REL_RR_LOG', 'VLF_LOG', 'LF_LOG', 'HF_LOG', 'TP_LOG', 'LF_HF_LOG',
       'RMSSD_LOG', 'SDRR_RMSSD_LOG', 'pNN25_LOG', 'pNN50_LOG', 'SD1_LOG',
       'KURT_YEO_JONSON', 'SKEW_YEO_JONSON', 'MEAN_REL_RR_YEO_JONSON',
       'SKEW_REL_RR_YEO_JONSON', 'LF_BOXCOX', 'HF_BOXCOX', 'SD1_BOXCOX',
       'KURT_SQUARE', 'HR_SQRT', 'MEAN_RR_MEAN_MEAN_REL_RR', 'SD2_LF', 'HR_LF',
       'HR_HF', 'HF_VLF', 'NasaTLX class', 'Condition Label', 'NasaTLX Label'])
# Добавить обратно идентификаторы субъектов
swell_train['subject_id']=subject_id_swell_train
swell_test['subject_id']=subject_id_swell_test

In [10]:
# Добавить обратно столбец состояния Condition label
swell_train_end = pd.concat([swell_train, label_swell_train], axis=1)
swell_test_end = pd.concat([swell_test, label_swell_test], axis=1)
swell_train_end.head()

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,subject_id,Condition Label
0,1174.144734,1220.305100,146.274795,20.453938,20.453875,7.151424,52.018538,21.266667,1.933333,14.467900,206.357241,-0.316083,-0.828116,0.000053,-0.000271,0.018127,0.007253,0.007253,2.499094,-0.316083,-0.828116,2667.302396,66.409695,1347.941161,33.560634,99.911668,1.191716,0.029671,0.088332,4016.435273,1131.093059,0.000884,4,1
1,895.342387,893.943175,56.174431,9.294380,9.294329,6.043914,67.278216,0.933333,0.000000,6.574276,79.170148,-0.451069,0.093802,0.000031,-0.001043,0.010565,0.005780,0.005780,1.827646,-0.451069,0.093802,684.553221,66.851455,328.488700,32.079241,96.774203,10.949590,1.069305,3.225797,1023.991510,30.000092,0.033333,17,1
2,764.218684,767.859965,77.160764,11.788299,11.787062,6.545539,79.342692,3.066667,0.000000,8.337493,108.802818,0.372389,-0.037786,-0.000234,-0.000238,0.015208,0.007286,0.007286,2.087272,0.372389,-0.037786,1470.421026,67.169385,698.527778,31.909011,97.192853,20.175042,0.921603,2.807147,2189.123845,34.623362,0.028882,18,1
3,854.776805,830.149445,84.950716,10.797540,10.796497,7.867599,70.867408,2.733333,0.200000,7.636824,119.895485,-0.835849,0.481628,-0.000154,-0.001338,0.012928,0.006824,0.006824,1.894481,-0.835849,0.481628,487.343787,46.827332,537.085656,51.606872,97.055263,16.295626,1.565795,2.944737,1040.725068,32.958885,0.030341,17,1
4,1093.290552,1121.317600,145.395568,17.550094,17.550090,8.284603,55.994337,16.066667,0.933333,12.413929,205.245309,-0.241385,-0.636697,-0.000016,-0.000583,0.016057,0.006023,0.006023,2.665965,-0.241385,-0.636697,3443.554474,78.304334,952.450230,21.658139,99.827029,1.650315,0.037527,0.172971,4397.655019,577.132367,0.001733,22,1


Загрузка данных датасета WESAD с гугл-диска

In [11]:
wesad_train =  pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/wesad-classification-hrv-train-dataset.csv")
wesad_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/wesad-classification-hrv-test-dataset.csv")

In [12]:
# Сопоставить объекты с соответствующими идентификаторами
subject_id_wesad_train=pd.DataFrame(wesad_train['subject id'].map({2: 26, 3: 27, 4: 28, 5: 29, 6: 30, 7: 31, 8: 32, 9: 33, 10: 34, 11: 35, 13:36, 14: 37, 15: 38, 16: 39, 17: 40}))
subject_id_wesad_test=pd.DataFrame(wesad_test['subject id'].map({2: 26, 3: 27, 4: 28, 5: 29, 6: 30, 7: 31, 8: 32, 9: 33, 10: 34, 11: 35, 13:36, 14: 37, 15: 38, 16: 39, 17: 40}))
subject_id_wesad_train

,subject id
0,31
1,28
2,36
3,29
4,29
...,...
108515,38
108516,38
108517,37
108518,36


In [13]:
# Показать все столбцы
wesad_train.columns

Index(['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR',
       'pNN25', 'pNN50', 'SD1', 'SD2', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR',
       'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'VLF_PCT',
       'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF',
       'HF_LF', 'MEAN_RR_LOG', 'MEAN_RR_SQRT', 'TP_SQRT', 'MEDIAN_REL_RR_LOG',
       'RMSSD_REL_RR_LOG', 'SDSD_REL_RR_LOG', 'VLF_LOG', 'LF_LOG', 'HF_LOG',
       'TP_LOG', 'LF_HF_LOG', 'RMSSD_LOG', 'SDRR_RMSSD_LOG', 'pNN25_LOG',
       'pNN50_LOG', 'SD1_LOG', 'KURT_YEO_JONSON', 'SKEW_YEO_JONSON',
       'MEAN_REL_RR_YEO_JONSON', 'SKEW_REL_RR_YEO_JONSON', 'LF_BOXCOX',
       'HF_BOXCOX', 'SD1_BOXCOX', 'KURT_SQUARE', 'HR_SQRT',
       'MEAN_RR_MEAN_MEAN_REL_RR', 'SD2_LF', 'HR_LF', 'HR_HF', 'HF_VLF',
       'subject id', 'condition', 'SSSQ class', 'SSSQ Label',
       'condition label'],
      dtype='object')

In [14]:
# Приведение индекса столбца выходных значений к одному виду
wesad_train=wesad_train.rename(columns={"condition label":"Condition Label"})
wesad_test=wesad_test.rename(columns={"condition label":"Condition Label"})

In [15]:
# Разделение данных WESAD на предикторы и переменные отклика
X_wesad_train=wesad_train.iloc[:, 0:32]
X_wesad_test=wesad_test.iloc[:, 0:32]
y_wesad_train=wesad_train['Condition Label']
y_wesad_test=wesad_test['Condition Label']

In [16]:
# Добавить обратно идентификаторы субъектов
X_wesad_train['subject_id']=subject_id_wesad_train
X_wesad_test['subject_id']=subject_id_wesad_test

In [17]:
# Присвоение состояниям субъектов метки "Condition Label": 1 - стресс, 0 - не стресс
# Замечание: 0-baseline; 1-amusement; 2-stress
label_wesad_train=pd.DataFrame(y_wesad_train.map({1: 0, 0: 0, 2: 1}))
label_wesad_test=pd.DataFrame(y_wesad_test.map({1: 0, 0: 0, 2: 1}))

In [18]:
# Добавить обратно столбец состояния Condition label
wesad_train_end = pd.concat([X_wesad_train, label_wesad_train], axis=1)
wesad_test_end = pd.concat([X_wesad_test, label_wesad_test], axis=1)
wesad_test_end.head()

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,subject_id,Condition Label
0,856.640559,855.196440,63.543133,15.490732,15.490690,4.102010,70.436793,9.75,0.0,10.967324,89.191801,0.773459,-0.102585,-0.000069,-0.002450,0.018102,0.009186,0.009185,1.970623,0.773459,-0.102585,1023.799120,48.247467,1065.508704,50.213069,97.025337,32.667024,1.539463,2.974663,2121.974847,32.617257,0.030659,32,0
1,911.890969,914.313935,49.739529,11.485676,11.485584,4.330570,65.996032,3.50,0.0,8.131731,69.870712,-0.447654,-0.178860,0.000030,-0.000799,0.012741,0.005881,0.005881,2.166519,-0.447654,-0.178860,1372.472455,73.366939,491.062152,26.250237,98.562600,7.161468,0.382824,1.437400,1870.696075,68.570042,0.014584,30,0
2,505.424097,500.162060,27.207534,5.217502,5.216857,5.214666,119.049905,0.00,0.0,3.693507,38.299581,-0.332671,0.431171,-0.000122,0.000281,0.010284,0.008260,0.008260,1.245031,-0.332671,0.431171,186.962752,45.288467,212.327622,51.432665,94.006990,13.536031,3.278867,5.993010,412.826404,15.686107,0.063751,35,1
3,976.606489,976.392520,93.145519,24.999690,24.999508,3.725867,62.014311,32.25,4.0,17.699516,130.533147,-0.596200,-0.127575,-0.000110,0.002974,0.026288,0.011375,0.011375,2.311054,-0.596200,-0.127575,2197.685692,49.348963,2249.469243,50.511761,99.725027,6.202481,0.139277,0.274973,4453.357416,362.672496,0.002757,40,0
4,740.323476,729.095405,73.676039,10.740166,10.739891,6.859860,81.833707,3.75,0.0,7.603784,103.915831,-0.375397,0.402807,-0.000144,0.001584,0.014044,0.006013,0.006013,2.335650,-0.375397,0.402807,3144.952405,85.098762,536.546893,14.518336,97.430403,14.150710,0.382902,2.569597,3695.650008,37.916606,0.026374,38,0


**Обучение модели**

In [19]:
# Соединить данные обоих датасетов в обучающие и тестовые данные
dfs_train = [wesad_train_end, swell_train_end]
dfs_test = [wesad_test_end, swell_test_end]
dfs_train=pd.concat(dfs_train, axis=0)
dfs_test=pd.concat(dfs_test, axis=0)
dfs_train

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,subject_id,Condition Label
0,860.758450,858.389130,75.161448,14.461513,14.460581,5.197343,70.242263,6.250000,0.750000,10.238012,105.800140,-0.870602,0.053677,-0.000165,-0.000286,0.016772,0.007971,0.007971,2.104142,-0.870602,0.053677,2013.584521,71.844187,769.968054,27.472266,97.572271,19.157841,0.683547,2.427729,2802.710416,40.190753,0.024881,31,0
1,1044.449374,1036.796500,82.209648,22.516679,22.515843,3.651056,57.792396,28.500000,2.250000,15.941094,115.163944,0.520593,0.578705,0.000211,-0.000638,0.021200,0.009560,0.009559,2.217618,0.520593,0.578705,1926.201330,50.004134,1923.619778,49.937117,99.882494,2.263030,0.058748,0.117506,3852.084138,850.019439,0.001176,28,0
2,510.072193,503.341590,32.094249,8.549018,8.547740,3.754145,118.059988,3.500000,0.000000,6.051753,44.982862,5.497297,1.623759,-0.000303,0.000003,0.016013,0.012763,0.012763,1.254616,5.497297,1.623759,462.314125,54.054568,354.014573,41.391997,90.089473,38.944294,4.553434,9.910527,855.272993,9.090281,0.110008,36,1
3,930.296969,940.394165,59.089578,7.816167,7.814190,7.559917,64.769049,0.000000,0.000000,5.532404,83.381947,0.565430,-0.596994,0.000169,-0.000397,0.008604,0.004306,0.004306,1.998056,0.565430,-0.596994,1227.245560,87.789136,165.025098,11.804818,96.674716,5.676306,0.406046,3.325284,1397.946964,29.072620,0.034397,29,0
4,692.067906,687.546285,47.584437,6.487698,6.487256,7.334564,87.096547,0.000000,0.000000,4.592942,67.137636,-0.376718,0.491406,0.000114,0.001556,0.009200,0.003206,0.003205,2.869915,-0.376718,0.491406,1306.992460,82.621635,270.453366,17.096732,98.379401,4.455165,0.281634,1.620599,1581.900991,60.705582,0.016473,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313305,713.800014,705.924890,60.701350,9.317147,9.316617,6.515015,84.655800,1.333333,0.000000,6.590041,85.591349,-0.137286,0.402129,0.000113,0.000191,0.013045,0.007027,0.007027,1.856364,-0.137286,0.402129,1133.123364,73.919624,374.272347,24.415763,93.617373,25.517066,1.664613,6.382627,1532.912777,14.667531,0.068178,14,0
313306,877.927568,918.630025,131.786551,15.977494,15.976853,8.248261,70.253466,9.733333,0.933333,11.301112,186.031381,0.003156,-0.970538,-0.000185,-0.000499,0.020224,0.011908,0.011908,1.698338,0.003156,-0.970538,1945.657049,67.301419,900.099062,31.134955,95.218060,45.203821,1.563627,4.781940,2890.959932,19.912013,0.050221,9,0
313307,807.683606,750.739005,207.227328,20.716765,20.716624,10.002881,78.629034,15.933333,3.533333,14.653754,292.697111,1.086939,1.121968,0.000053,-0.000285,0.028608,0.016512,0.016512,1.732547,1.086939,1.121968,8069.311303,84.305491,1369.444056,14.307498,91.162444,132.757936,1.387011,8.837556,9571.513295,10.315346,0.096943,10,1
313308,779.766291,786.363725,66.561657,13.706979,13.706955,4.856041,77.534374,4.800000,0.600000,9.695516,93.631754,-0.145431,-0.316590,-0.000020,-0.001003,0.017910,0.008320,0.008320,2.152774,-0.145431,-0.316590,986.738916,48.466016,1023.740666,50.283445,97.573369,25.460231,1.250539,2.426631,2035.939813,40.209402,0.024870,5,0


In [20]:
# Показать количество экземпляров для каждого объекта по параметру 'subject_id'
dfs_train['subject_id'].value_counts()

16    17223
6     16713
19    15998
14    15757
12    15716
10    15700
4     15621
2     15554
1     15413
17    15045
18    14955
5     14373
7     14109
21    13272
24    12967
13    12822
3     12742
9     12706
22    12181
25    11844
20    11810
23    10789
34     7481
40     7480
38     7320
35     7311
36     7308
39     7277
37     7233
29     7204
32     7198
30     7185
31     7176
28     7165
33     7137
27     7094
26     6951
Name: subject_id, dtype: int64

In [21]:
# показать количество экземпляров для каждого объекта по параметру 'Condition Label' для тестовой выборки
dfs_test['Condition Label'].value_counts()

0    30852
1    21877
Name: Condition Label, dtype: int64

In [22]:
# показать количество экземпляров для каждого объекта по параметру 'Condition Label' для тренировочной выборки
dfs_train['Condition Label'].value_counts()

0    245627
1    176203
Name: Condition Label, dtype: int64

In [23]:
# Разделить тренировочную и тестовую выборки на предикторы и отклики
X_train, y_train = dfs_train.iloc[:, :-2], dfs_train.iloc[:, -1]
X_test, y_test = dfs_test.iloc[:, :-2], dfs_test.iloc[:, -1]

In [32]:
# Выбрать 12 предикторов с помощью mRMR
features = mrmr_classif(X=X_train, y=y_train, K=12)
print(features)
X_train_features=X_train[features]
X_test_features=X_test[features]

100%|██████████| 12/12 [00:05<00:00,  2.36it/s]

['KURT', 'VLF', 'MEAN_REL_RR', 'SDSD_REL_RR', 'KURT_REL_RR', 'TP', 'MEDIAN_REL_RR', 'pNN25', 'LF', 'SD2', 'RMSSD_REL_RR', 'SDRR']


In [25]:
# Создать функции для оценки качества моделей
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def accuracy_and_conf_matrix(model, y_true, inputs):
    y_pred = model.predict(inputs)
    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return accuracy, conf_matrix

from sklearn.metrics import classification_report

def class_report(model, y_true, inputs):
    y_pred = model.predict(inputs)
    report = classification_report(y_true, y_pred)
    return report

In [46]:
# Обучить модель на данных
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

adaboost_clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=3, class_weight='balanced'),
    n_estimators=300,
    learning_rate=0.5,
    random_state=42
)

adaboost_clf.fit(X_train_features, y_train)

KeyboardInterrupt: 

In [35]:
# Оценить точность модели
accuracy, conf_matrix = accuracy_and_conf_matrix(adaboost_clf, y_train, X_train_features)
print('adaboost_clf_train_accuracy', accuracy)
print('\n\n', conf_matrix)

accuracy, conf_matrix = accuracy_and_conf_matrix(adaboost_clf, y_test, X_test_features)
print('adaboost_clf_test_accuracy', accuracy)
print('\n\n', conf_matrix)

adaboost_clf_train_accuracy 0.9679918450560653


 [[236616   9011]
 [  4491 171712]]
adaboost_clf_test_accuracy 0.9665269585996321


 [[29684  1168]
 [  597 21280]]


In [36]:
# Сериализовать данные
import pickle

file = "adaboostpickled.tmp"
with open("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/" + file, "wb") as outfile:
    pickled_model = pickle.dump(adaboost_clf, outfile)
print("Written object", pickled_model)

Written object None


In [38]:
# Десериализация и проверка работоспособности модели
with open("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/" + file, "rb") as infile:
    pickled_model = pickle.load(infile)

accuracy, conf_matrix = accuracy_and_conf_matrix(pickled_model, y_test, X_test_features)
print('adaboost_clf_test_accuracy', accuracy)
print('\n\n', conf_matrix)

adaboost_clf_test_accuracy 0.9665269585996321


 [[29684  1168]
 [  597 21280]]


In [39]:
print(pickled_model.predict(X_test_features))

[0 0 1 ... 0 0 1]
